# Day 16

## Part 1

- The beam enters at the top left from the left
- Tiles have these effects:
    - `.` leaves it unchanged
    - `/` and `\` reflects the beam 90 degrees
    - `|` and `-` do nothing if the beam enters from the narrow side. If the beam enters from the flat side, 2 beams are emitted out of the ends.
- How many tiles have the beam passing through?

In [2]:
from matplotlib import pyplot as plt
from tqdm import tqdm

from advent_of_code_utils.advent_of_code_utils import (
    ParseConfig, parse_from_file, markdown, plot_grid
)

In [ ]:
EMPTY = 0
SLASH = 1
BACK = 2
V_SPLIT = 3
H_SPLIT = 4

def parse_tile(tile: str) -> int:
    """encodes the tile into an integer"""
    match tile:
        case '.':
            return EMPTY
        case '/':
            return SLASH
        case '\\':
            return BACK
        case '|':
            return V_SPLIT
        case '-':
            return H_SPLIT